A person from marketing is preparing a campaign and is working its budget allocations per city using [an Excel file](https://www.dropbox.com/s/ssjxk3j58iuwfai/campaign.xlsx?dl=1).
The person would like to augment the data using external sources in order to get a better plan.

In [8]:
%load_ext raw_magic

The raw_magic extension is already loaded. To reload it, use:
  %reload_ext raw_magic


In [9]:
%buckets_register raw-tutorial

API error: S3 credentials already exists


In [3]:
%%view campaign
SELECT *
FROM read_excel("s3://raw-tutorial/ipython-demos/campaign.xlsx", cache := interval "1 day")

In [4]:
%%query
SELECT *
FROM campaign
ORDER BY Location

Location,PrintMedia,SocialMedia,Radio,Television,Total
Basel,102.0,52.0,102.0,202.0,458.0
Fribourg,105.0,55.0,105.0,205.0,470.0
Geneve,100.0,50.0,100.0,200.0,450.0
Lausanne,103.0,53.0,103.0,203.0,462.0
Luzern,104.0,54.0,104.0,204.0,466.0
Neuchatel,106.0,56.0,106.0,206.0,474.0
Zug,107.0,57.0,107.0,207.0,478.0
Zurich,101.0,51.0,101.0,201.0,454.0


CDR information could be used to get interesting statistics about phone calls in regions where the campaign is going to take place. It is found in large text log files.
```
59cc3bd800e83d965300166d|928147382718783|928147382718999|2016-07-05 00:01:41|2016-07-05 00:01:41|call
59cc3bd700e83d9653001614|928147382720125|928147382719444|2016-07-05 00:03:25|2016-07-05 00:12:17|call
59cc3bf2f0b8ac50d1001781|928147382719070|928147382720180|2016-07-05 00:04:52|2016-07-05 00:04:54|SMS
59cc3bd800e83d965300164e|928147382718877|928147382719907|2016-07-05 00:07:23|2016-07-05 00:07:24|SMS
59cc3bdb00e83d9653001712|928147382719498|928147382719102|2016-07-05 00:07:29|2016-07-05 00:07:29|SMS
59cc3becf0b8ac50d100165b|928147382719541|928147382720077|2016-07-05 00:08:50|2016-07-05 00:11:00|call
59cc3bd900e83d965300168b|928147382719249|928147382719136|2016-07-05 00:12:25|2016-07-05 00:12:25|SMS
59cc3bf1f0b8ac50d100175a|928147382718251|928147382718436|2016-07-05 00:12:57|2016-07-05 00:16:58|call
59cc3bda00e83d96530016d9|928147382718421|928147382719367|2016-07-05 00:13:13|2016-07-05 00:13:15|SMS
59cc3bd900e83d9653001687|928147382719942|928147382718754|2016-07-05 00:13:21|2016-07-05 00:17:49|call
59cc3becf0b8ac50d1001658|928147382718600|928147382718599|2016-07-05 00:13:49|2016-07-05 00:14:01|call
```
CSV files can be easily loaded in RAW (using `read_csv`). As the files do not have a header, it's added using the `INTO` keyword, and the query also takes care of parsing timestamps.

In [5]:
%%view cdr
    SELECT *
    FROM read_csv("s3://raw-tutorial/ipython-demos/CDR.txt", cache := interval "1 day")
    INTO (
        tower: _1, caller: _2, callee: _3,
        start: to_timestamp(_4, "yyyy-MM-dd HH:mm:ss"),
        end: to_timestamp(_5, "yyyy-MM-dd HH:mm:ss"),
        type: _6
    )

In [6]:
%%query
SELECT *
FROM cdr
LIMIT 20

tower,caller,callee,start,end,type
59cc3bd800e83d965300166d,928147382718783,928147382718999,2016-07-05 00:01:41,2016-07-05 00:01:41,call
59cc3bd700e83d9653001614,928147382720125,928147382719444,2016-07-05 00:03:25,2016-07-05 00:12:17,call
59cc3bf2f0b8ac50d1001781,928147382719070,928147382720180,2016-07-05 00:04:52,2016-07-05 00:04:54,SMS
59cc3bd800e83d965300164e,928147382718877,928147382719907,2016-07-05 00:07:23,2016-07-05 00:07:24,SMS
59cc3bdb00e83d9653001712,928147382719498,928147382719102,2016-07-05 00:07:29,2016-07-05 00:07:29,SMS
59cc3becf0b8ac50d100165b,928147382719541,928147382720077,2016-07-05 00:08:50,2016-07-05 00:11:00,call
59cc3bd900e83d965300168b,928147382719249,928147382719136,2016-07-05 00:12:25,2016-07-05 00:12:25,SMS
59cc3bf1f0b8ac50d100175a,928147382718251,928147382718436,2016-07-05 00:12:57,2016-07-05 00:16:58,call
59cc3bda00e83d96530016d9,928147382718421,928147382719367,2016-07-05 00:13:13,2016-07-05 00:13:15,SMS
59cc3bd900e83d9653001687,928147382719942,928147382718754,2016-07-05 00:13:21,2016-07-05 00:17:49,call


In [7]:
%%query
SELECT type, COUNT(*) AS n
FROM cdr
GROUP BY type

type,n
SMS,100865
call,99135


It can be interesting to focus on "power users" of the provider, like those which spend a large amount of time calling.

In [8]:
%%view power_users
SELECT caller
FROM cdr
GROUP BY caller
HAVING SUM(end-start) > interval "5 hours"

In [9]:
%%query
SELECT *
FROM power_users
LIMIT 10

long
928147382718261
928147382718271
928147382718283
928147382718293
928147382718297
928147382718373
928147382718371
928147382718381
928147382718383
928147382718393


Now that the power users have been identified, it's possible to restrict the CDR to these users.

In [10]:
%%query
SELECT COUNT(*)
FROM cdr
WHERE caller IN power_users

long
33696


In [11]:
%%view cdr_power_users
SELECT *
FROM cdr
WHERE caller in power_users

The CDR info is now filtering the power users. We'd like to see where the phone calls have been done. The `tower` field is the internal identifier of the antennas of the provider. The location of antennas is known but it requires to access another external table.

In [12]:
%%query
SELECT *
FROM cdr_power_users
LIMIT 10

tower,caller,callee,start,end,type
59cc3bd800e83d965300166d,928147382718783,928147382718999,2016-07-05 00:01:41,2016-07-05 00:01:41,call
59cc3becf0b8ac50d100165b,928147382719541,928147382720077,2016-07-05 00:08:50,2016-07-05 00:11:00,call
59cc3bd900e83d9653001687,928147382719942,928147382718754,2016-07-05 00:13:21,2016-07-05 00:17:49,call
59cc3bd800e83d9653001641,928147382718293,928147382719203,2016-07-05 00:22:48,2016-07-05 00:25:54,call
59cc3beef0b8ac50d10016be,928147382719830,928147382718790,2016-07-05 00:24:41,2016-07-05 00:32:46,call
59cc3beff0b8ac50d1001709,928147382719315,928147382718634,2016-07-05 00:27:35,2016-07-05 00:27:35,call
59cc3becf0b8ac50d1001670,928147382718364,928147382718334,2016-07-05 00:27:46,2016-07-05 00:32:35,call
59cc3beaf0b8ac50d100160b,928147382718887,928147382718813,2016-07-05 00:30:01,2016-07-05 00:30:31,call
59cc3bf2f0b8ac50d100178a,928147382720305,928147382719218,2016-07-05 00:48:04,2016-07-05 00:50:43,call
59cc3beaf0b8ac50d10015f3,928147382720259,928147382719632,2016-07-05 00:54:55,2016-07-05 01:03:13,call


The antennas as found in an operational database. The `id` field is the one we need to join on, which will permit to extract the `city` where the antennas are located.

In [13]:
%%view towers
SELECT *
FROM read_csv[record(id: string, name: string, city: string, lat:double, long:double)](
    "s3://raw-tutorial/ipython-demos/antennas.csv",
    delimiter := "|",
    cache := interval "1 week"
)

In [14]:
%%query
SELECT *
FROM towers
WHERE city = "Lausanne"
LIMIT 10

id,name,city,lat,long
59cc3beef0b8ac50d10016d3,MGIN Morgins,Lausanne,46.24133379,6.86238354
59cc3bf1f0b8ac50d1001779,VLOR Vallorbe,Lausanne,46.71404927,6.36789015
59cc3beaf0b8ac50d10015f2,"AUBS Auberson, L'",Lausanne,46.83388084,6.5001323
59cc3bf0f0b8ac50d100171c,ROMO Romainmôtier,Lausanne,46.69144782,6.46285045
59cc3bf1f0b8ac50d1001778,VLOP Vallorbe Pralioux Dessous,Lausanne,46.717556,6.36578002
59cc3bf0f0b8ac50d1001735,SCRX Ste-Croix,Lausanne,46.81262229,6.49455377
59cc3beef0b8ac50d10016c2,LSNR Lausanne Renens,Lausanne,46.533287,6.58625368
59cc3bedf0b8ac50d100169d,JOUX Vallée de Joux,Lausanne,46.67061405,6.33385672
59cc3beaf0b8ac50d10015fb,"BAYA Bayards, Les",Lausanne,46.91372984,6.50685167
59cc3bebf0b8ac50d100162e,CHPY Champéry,Lausanne,46.16561916,6.86297639


In [15]:
%%query
SELECT DISTINCT city
FROM towers

string
Luzern
Sarnen
Schwyz
Herisau
Sitten
Zürich
Schaffhausen
Fribourg
Lausanne
Appenzell


Going back to our CDR data restricted to power users, we add a `city` field to the table which consists in picking the matching row found in the antenna table.

In [16]:
%%view cdr_power_by_city
SELECT tower, caller, callee, start, end, type,
    CFIRST(
        SELECT city
        FROM towers
        WHERE id = tower
    ) AS city
FROM cdr_power_users

Now the CDR rows have the city field.

In [17]:
%%query
SELECT *
FROM cdr_power_by_city
LIMIT 10

tower,caller,callee,start,end,type,city
59cc3bd800e83d965300166d,928147382718783,928147382718999,2016-07-05 00:01:41,2016-07-05 00:01:41,call,Altdorf
59cc3becf0b8ac50d100165b,928147382719541,928147382720077,2016-07-05 00:08:50,2016-07-05 00:11:00,call,Neuchâtel
59cc3bd900e83d9653001687,928147382719942,928147382718754,2016-07-05 00:13:21,2016-07-05 00:17:49,call,Aarau
59cc3bd800e83d9653001641,928147382718293,928147382719203,2016-07-05 00:22:48,2016-07-05 00:25:54,call,Frauenfeld
59cc3beef0b8ac50d10016be,928147382719830,928147382718790,2016-07-05 00:24:41,2016-07-05 00:32:46,call,Stans
59cc3beff0b8ac50d1001709,928147382719315,928147382718634,2016-07-05 00:27:35,2016-07-05 00:27:35,call,Solothurn
59cc3becf0b8ac50d1001670,928147382718364,928147382718334,2016-07-05 00:27:46,2016-07-05 00:32:35,call,Solothurn
59cc3beaf0b8ac50d100160b,928147382718887,928147382718813,2016-07-05 00:30:01,2016-07-05 00:30:31,call,Neuchâtel
59cc3bf2f0b8ac50d100178a,928147382720305,928147382719218,2016-07-05 00:48:04,2016-07-05 00:50:43,call,Sarnen
59cc3beaf0b8ac50d10015f3,928147382720259,928147382719632,2016-07-05 00:54:55,2016-07-05 01:03:13,call,Bellinzona


In [18]:
%%query
SELECT city, COUNT(*)
FROM cdr_power_by_city
GROUP BY city
LIMIT 10

city,_2
Neuchâtel,1431
Sankt Gallen,431
Delémont,1120
Schaffhausen,248
Frauenfeld,855
Liestal,1140
Luzern,106
Genève,38
Zürich,801
Schwyz,712


Since the CDR contain the IMEI of callers, we can arrange IMEI's of callers by city. We still need to match these IMEI's to customers to get interesting statistics. This will be obtained thanks to a third external data source, the customer management system table (CMS).

In [19]:
%%query
SELECT city, (SELECT DISTINCT caller FROM partition LIMIT 10)
FROM cdr_power_by_city
GROUP BY city
LIMIT 2

CMS info is found in a second operation database table. It contains personal info about customers.

In [20]:
%%view cms
SELECT *
FROM read_csv[record(imei: long, first_name: string, name: string, gender: string, age: int, address:string)](
    "s3://raw-tutorial/ipython-demos/cms.csv",
    delimiter := "|",
    cache := interval "1 day"
)

In [21]:
%%query
SELECT *
FROM cms
LIMIT 10

imei,first_name,name,gender,age,address
928147382718234,Stephannie,Ashmore,F,61,19 Mosinee Road
928147382718235,Norton,Sture,M,54,37632 Eagan Center
928147382718236,Teador,Norment,M,23,5 Mcguire Drive
928147382718237,Rozelle,Reinink,F,45,403 Towne Center
928147382718238,Suzi,Beaton,F,44,2443 Columbus Alley
928147382718239,Anastassia,Nicandro,F,28,9 Loomis Street
928147382718240,Jojo,Hyde,F,21,8 Lukken Junction
928147382718241,Meredith,Leffler,M,67,5716 Longview Junction
928147382718242,Elisabet,Misko,F,59,86533 Memorial Point
928147382718243,Rebbecca,Ress,F,27,21998 Gulseth Circle


Since we want to extract info from this table for each of the power users found in the various cities, we prepare a helper function. It takes an IMEI as a parameter and finds a match in the CMS table.

In [22]:
%%view customer_info
\imei:long ->
  CFIRST (
    SELECT *
    FROM cms c_info
    WHERE c_info.imei = imei
  )

In [23]:
%%query
customer_info(928147382718236)

imei,first_name,name,gender,age,address
928147382718236,Teador,Norment,M,23,5 Mcguire Drive


We can now replace the internal IMEI reported in the CDR by the matching customer information.

In [24]:
%%query
SELECT city, SELECT customer_info(c) FROM c IN (SELECT DISTINCT caller FROM partition LIMIT 10)
FROM cdr_power_by_city
GROUP BY city
LIMIT 2

In [25]:
%%view power_users_by_city
SELECT city, (
    SELECT customer_info(caller) FROM caller IN (SELECT DISTINCT caller FROM partition)) AS users
FROM cdr_power_by_city
GROUP BY city

In [26]:
%%query
SELECT *
FROM power_users_by_city
WHERE city = "Lausanne"

Now that we have all the information we need, let's finalize a statistics table from this view.

In [27]:
%%view city_stats
SELECT city,
    (SELECT AVG(age) FROM users WHERE gender = "F") AS f,
    (SELECT AVG(age) FROM users WHERE gender = "M") AS m
FROM power_users_by_city

In [28]:
%%query
SELECT *
FROM city_stats

city,f,m
Neuchâtel,38,47
Sankt Gallen,51,49
Delémont,44,53
Schaffhausen,36,50
Frauenfeld,44,42
Liestal,48,44
Luzern,null,28
Genève,39,null
Zürich,41,45
Schwyz,49,40


Back to our Excel file. We can project the content of the Excel file and query the `city_stats` view we just built. Now the rows are added the statistics information we wanted.

Unfortunately, there are little typos in the Excel file (the author didn't use accents while the antennas table does contain accents) and the match is failing for certain cities.

In [29]:
%%query
SELECT Location, PrintMedia, SocialMedia, Radio, Television, Total,
   CFIRST(SELECT * FROM city_stats WHERE city = Location) AS stats
FROM campaign

A solution: match the city name by similarity and pick the closest match. This permits to solve the problem.

In [30]:
%%query
SELECT Location, PrintMedia, SocialMedia, Radio, Television, Total,
   CFIRST(
       SELECT *
       FROM city_stats
       ORDER BY levenshtein_distance(city,Location)
       ASC) AS stats
FROM campaign